In [4]:
import requests
import pandas as pd
from config import tmdb_key
from pprint import pprint as pprint

In [5]:
base_params = dict(api_key=tmdb_key)
year = 2019
page = 1
listing_params = dict(primary_release_year=year,region='US', sort_by='revenue.desc', include_adult='false', language='en-US', page='1')
listing_params.update(base_params)
response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
imdb_films = response.json()['results']


In [6]:
rows = []

restart = True
page_limit = True
while restart == True:
   while page_limit == True:
      for film in imdb_films:
         film_revenue = requests.get('https://api.themoviedb.org/3/movie/{}'.format(film['id']), params=listing_params)
         film_revenue = film_revenue.json()
         try:
            row = dict(title=film['title'], revenue=film_revenue['revenue'], Year=listing_params['primary_release_year'],\
                     VoteAVG=film['vote_average'], VoteCT=film['vote_count'], Popularity=film['popularity'], Date=film['release_date'],\
                        Genre=film['genre_ids'][0])
         except:
            pass   
         rows.append(row)
      #MODIFY page == x: to increase or decrease sample size
      if page == 10: 
         page_limit = False
      else:
         page += 1
         listing_params.update({'page':page})
         response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
         imdb_films = response.json()['results']
   if year == 2023: 
      restart = False
   else:
      page_limit = True
      page = 1
      year += 1
      listing_params.update({'primary_release_year':year})
      response = requests.get('https://api.themoviedb.org/3/discover/movie', params=listing_params)
      imdb_films = response.json()['results']
      
df = pd.DataFrame(rows, columns=['title', 'revenue', 'Year','VoteAVG', 'VoteCT', 'Popularity', 'Date', 'Genre'])
df = df.set_index('Year')
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
#Copy df
df_copy = df.copy()

In [8]:
# a = [28,12,16,35,80,99,18,10751,14,36,27,10402,9648,10749,878,10770,53,10752,37]
# b = ['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','History',\
#                      'Horror','Music','Mystery','Romance','Science', 'Fiction','TV Movie','War','Western']

# data = {}

# for num,genre in zip(a,b):
#     data[num] = genre
# data

In [9]:
data = pd.DataFrame(
       {
              'A': [28,12,16,35,80,99,18,10751,14,36,27,10402,9648,10749,878,10770,53,10752,37],
              'B':['Action','Adventure','Animation','Comedy','Crime','Documentary','Drama','Family','Fantasy','History',\
                     'Horror','Music','Mystery','Romance','Science', 'Fiction','TV Movie','War','Western']
       }
)
map_dict = {28:'Action', 12:'Adventure'}
map_dict = {x:data['B'].tolist()[i] for i,x in enumerate(data['A'].tolist())}
df_copy['Genre'] = df_copy['Genre'].map(map_dict)
df_copy = df_copy.reset_index()
df_copy.head()

,Year,title,revenue,VoteAVG,VoteCT,Popularity,Date,Genre
0,2019,Avengers: Endgame,2800000000,8.3,24042,148.904,2019-04-26,Adventure
1,2019,The Lion King,1663075401,7.1,9397,81.710,2019-07-19,Adventure
2,2019,Frozen II,1450000000,7.3,9114,104.249,2019-11-22,Family
3,2019,Spider-Man: Far From Home,1131927996,7.4,14610,76.017,2019-07-02,Action
4,2019,Captain Marvel,1131416446,6.8,14765,129.554,2019-03-08,Action


In [11]:
df_copy.to_csv("output_data/tmdb_movies.csv", index_label="Year")